In [4]:
import arcgis
from arcgis.gis import GIS

In [10]:
from copy import deepcopy
from datetime import datetime
from IPython.display import HTML
import json
import pandas as pd
from arcgis.gis import GIS
import arcgis.network as network
import arcgis.geocoding as geocoding
import requests
import json
import sys
from IPython.display import FileLink, FileLinks


my_gis = GIS('https://www.arcgis.com', user_name, password)

In [11]:
route_service_url = my_gis.properties.helperServices.route.url
route_service_url

'https://route.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World'

In [12]:
route_service = network.RouteLayer(route_service_url, gis=my_gis)
route_service

<RouteLayer url:"https://route.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World">

In [13]:
#Route Blue
blue_stops = '''
-76.943882,38.987431;
-76.945355,38.988067;
-76.945293,38.98869;
-76.942127,38.989256;
-76.940239,38.986649;
-76.940156,38.984922;
-76.939149,38.982427;
-76.937091,38.985274;
-76.929719,38.9906131;
-76.9340225,38.992351;
-76.9341941,38.9905204;
-76.939524,38.987735;
-76.943882,38.987431;
'''

campus_connector = '''
-76.943945,38.98761;
-76.94678,38.985277;
-76.945186,38.983749; 
-76.944942,38.982413;
-76.939481,38.978162;
-76.937771,38.979462;
-76.939114,38.982511;
-76.940049,38.98454;
-76.940112,38.986373;
-76.9406072,38.9903818;
-76.940732,38.991492;
-76.939835,38.993664;
-76.937715,38.993702;
-76.937647,38.995334;
-76.940504,38.997819;
-76.944723,39.001073;
-76.9434,39.002685;
-76.942541,39.003083;
-76.94343,39.002056;
-76.944723,39.001073;
-76.940409,38.997908;
-76.937462,38.995357;
-76.937479,38.993628;
-76.939932,38.993483;
-76.940647,38.991274;
-76.940772,38.989522;
-76.943945,38.98761;
'''

route_layer = network.RouteLayer(route_service_url, gis=my_gis)
result = route_layer.solve(stops=blue_stops, 
                           directions_language='en-US', return_routes=True,
                           return_stops=True, return_directions=False,
                           return_barriers=False, return_polygon_barriers=False,
                           return_polyline_barriers=False)

result2 = route_layer.solve(stops=campus_connector, 
                           directions_language='en-US', return_routes=True,
                           return_stops=True, return_directions=False,
                           return_barriers=False, return_polygon_barriers=False,
                           return_polyline_barriers=False)


In [14]:
my_map = my_gis.map('University of Maryland', zoomlevel=16)
my_map

MapView(layout=Layout(height='400px', width='100%'))

In [15]:
my_map.clear_graphics()
stop_count = result['routes']['features'][0]['attributes']['StopCount']
travel_time = result['routes']['features'][0]['attributes']['Total_TravelTime']
distance = result['routes']['features'][0]['attributes']['Total_Miles']

data = [('Number of stops', stop_count),
        ('Total travel time', '{0:.2f} min'.format(travel_time)),
        ('Total travel distance', '{0:.2f} mi'.format(distance))]

df = pd.DataFrame.from_records(data)
styles = [    
    dict(selector="td", props=[("padding", "2px")]),
    dict(selector='.row_heading, .blank', props=[('display', 'none;')]),
    dict(selector='.col_heading, .blank', props=[('display', 'none;')])]

symbol = {
    "type": "esriSLS",
    "style": "esriSLSSolid",
    "color": [0,191,255,90],
    "width": 4
}

popup_route = {"title": "Route", 
               "content": df.style.set_table_styles(styles).render()}
popup_stop = {"title": "Stop {}", 
              "content": df.style.set_table_styles(styles).render()}

my_map.draw(result['routes']['features'][0]['geometry'], popup_route, symbol)

for stop in result['stops']['features']:
    address = geocoding.reverse_geocode(stop['geometry'])['address']['Match_addr']
    my_map.draw(stop['geometry'], 
                {"title": "Stop {}".format(stop['attributes']['Sequence']), 
                 "content": address})
my_map.zoom = 15

In [17]:
stop_count = result2['routes']['features'][0]['attributes']['StopCount']
travel_time = result2['routes']['features'][0]['attributes']['Total_TravelTime']
distance = result2['routes']['features'][0]['attributes']['Total_Miles']

data = [('Number of stops', stop_count),
        ('Total travel time', '{0:.2f} min'.format(travel_time)),
        ('Total travel distance', '{0:.2f} mi'.format(distance))]

df = pd.DataFrame.from_records(data)
styles = [    
    dict(selector="td", props=[("padding", "2px")]),
    dict(selector='.row_heading, .blank', props=[('display', 'none;')]),
    dict(selector='.col_heading, .blank', props=[('display', 'none;')])]

symbol = {
    "type": "esriSLS",
    "style": "esriSLSSolid",
    "color": [128,0,128,90],
    "width": 4
}

popup_route = {"title": "Route", 
               "content": df.style.set_table_styles(styles).render()}
popup_stop = {"title": "Stop {}", 
              "content": df.style.set_table_styles(styles).render()}

my_map.draw(result2['routes']['features'][0]['geometry'], popup_route, symbol)

for stop in result2['stops']['features']:
    address = geocoding.reverse_geocode(stop['geometry'])['address']['Match_addr']
    my_map.draw(stop['geometry'], 
                {"title": "Stop {}".format(stop['attributes']['Sequence']), 
                 "content": address})
my_map.zoom = 15